# Agent: People (Phase 1)

This notebook runs a single **People agent** with local random-walk simulation only (no MQTT yet).

In [ ]:
# Cell purpose: import dependencies and load project configuration.
from __future__ import annotations

import random
import time

from simulated_city.config import load_config
from simulated_city.models import Person, PersonState
from simulated_city.movement import apply_boundary_bounce, random_walk_step

config = load_config()
print(f"Loaded base topic from config: {config.mqtt.base_topic}")

In [ ]:
# Cell purpose: define simulation constants and initialize 5 people.
rng = random.Random(42)

PEOPLE_COUNT = 5
STEP_DISTANCE = 1.2
MAX_TURN_DEG = 45.0
TICK_SECONDS = 1.0
TOTAL_TICKS = 20

# Phase 1 uses a local abstract coordinate plane (not geospatial mapping yet).
MIN_X, MAX_X = 0.0, 100.0
MIN_Y, MAX_Y = 0.0, 100.0

names = ["Alex", "Sam", "Jordan", "Taylor", "Casey", "Riley", "Morgan", "Avery", "Parker", "Quinn"]
colors = ["red", "blue", "green", "orange", "purple", "teal", "pink", "brown", "gray", "black"]

people: list[Person] = []
for index in range(PEOPLE_COUNT):
    person = Person(
        person_id=f"person_{index + 1}",
        name=names[index],
        color=colors[index],
        x=rng.uniform(MIN_X, MAX_X),
        y=rng.uniform(MIN_Y, MAX_Y),
        heading_deg=rng.uniform(0.0, 360.0),
        state=PersonState.RANDOM_WALK,
    )
    people.append(person)

print(f"Initialized {len(people)} people for random-walk simulation.")

In [ ]:
# Cell purpose: run the local simulation loop and print per-tick state output.
for tick in range(1, TOTAL_TICKS + 1):
    print(f"\nTick {tick}")
    for person in people:
        x1, y1, heading1 = random_walk_step(
            person.x,
            person.y,
            person.heading_deg,
            step_distance=STEP_DISTANCE,
            max_turn_deg=MAX_TURN_DEG,
            rng=rng,
        )
        x2, y2, heading2 = apply_boundary_bounce(
            x1, y1, heading1, MIN_X, MAX_X, MIN_Y, MAX_Y
        )
        person.x = x2
        person.y = y2
        person.heading_deg = heading2

        log = person.to_log_dict()
        print(
            f"- {log['person_id']} ({log['name']}, {log['color']}): "
            f"x={log['x']}, y={log['y']}, heading={log['heading_deg']}, state={log['state']}"
        )

    time.sleep(TICK_SECONDS)

print("\nPhase 1 people-agent simulation complete.")